In [1]:
import pandas as pd
import os
database_url = os.environ.get('DATABASE_URL')
import psycopg2
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String

In [2]:
from lib.data_model_lib import *

In [3]:
from flask import Flask
from sqlalchemy import Column, Integer, String, create_engine, select
from sqlalchemy.orm import sessionmaker
from werkzeug.security import generate_password_hash

#app = Flask(__name__)
#app.config['SQLALCHEMY_DATABASE_URI'] = database_url
#app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
#db.init_app(app)


##########

engine = create_engine(database_url, echo=False)
#Base = declarative_base(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()

conn = session.bind


In [4]:
df_query = select([User])

df_data = pd.read_sql(df_query, con = conn)
df_data

,id,email,password,name,mobile,broker_id,broker_name,api_key,api_secret,api_token,session_id,is_active,is_admin
0,2,mailme@suhan.in,sha256$YsLjJSJY$71213b96a447e14e492c52cbf07e09...,Saswati Saha,None,None,None,None,None,None,None,False,False
1,1,suhansaha@gmail.com,sha256$4wgMT9w0$1638131fa1d88330e543a726d3669b...,Suhan Saha,+917984183480,YM1755,ZERODHA,b2w0sfnr1zr92nxm,jtga2mp2e5fn29h8w0pe2kb722g3dh1q,None,None,True,True


In [7]:
# Update user table
user = session.query(User).filter_by(email='suhansaha@gmail.com').first()
user.name = 'Suhan Saha'
user.mobile = '+917984183480'
user.broker_name = 'ZERODHA'
user.broker_id = 'YM1755'
user.api_key = 'b2w0sfnr1zr92nxm'
user.api_secret = 'jtga2mp2e5fn29h8w0pe2kb722g3dh1q'
session.commit()

In [29]:
# Update algo
def update_algo(name, algo_str, user_id):
    algo = session.query(Algos).filter(Algos.title==name, Algos.user_id==user_id).first()
    if algo == None:
        new_algo = Algos(title=name, algo=algo_str, user_id=user_id)
        session.add(new_algo)
        session.commit()
    else:
        algo.algo = algo_str
        session.commit()
        
def update_algo1(name, algo_str, user_id):
    user = session.query(User).filter(User.id==user_id).first()
    user.algos = [Algos(title=name, algo=algo_str)]
    session.commit()
    
def load_algo_to_db(filename, user_id):
    algo = open('algo/'+filename+'.txt')
    algo_str = algo.read()
    print(algo_str)
    #my_cache.hset('algos', filename,algo_str)
    update_algo(filename, algo_str, user_id)
    
load_algo_to_db('bull_squeeze', 1)

roc = ROC(CLOSE, 10)
bbh, bbm, bbl = BBANDS(CLOSE)

delbb = bbh - bbl

#sell = (REF(roc,0) > 0.1) & (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
#buy = (REF(roc,0) < -0.1) & (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
buy = delbb > 2.1 * REF(delbb, 3)
sell = delbb < 0.4 * REF(delbb, 3)


In [17]:
algos = session.query(User).filter(User.id==1).first().algos

In [28]:
import datetime as dt

In [39]:
"{}-{}".format(int(dt.datetime.now().timestamp()*1000000),1)

'1591476589829133-1'

## Scratchpad

In [4]:
df_query = select([User])

df_data = pd.read_sql(df_query, con = conn)
df_data

,id,email,password,name,mobile,broker_id,broker_name,api_key,api_secret,api_token,session_id
0,1,suhansaha@gmail.com,sha256$ZPzKNGp4$487b1d21cd69cebcd9dd7022bbba4d...,Suhan Saha,+917984183480,YM1755,ZERODHA,b2w0sfnr1zr92nxm,jtga2mp2e5fn29h8w0pe2kb722g3dh1q,None,None


In [1]:
import pandas as pd